In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime, timezone, timedelta
import pytz
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import requests
import ScraperFC as sfc
import random
import numpy as np
import os
import urllib.parse
from time import perf_counter
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.set_option('display.max_rows', 50)
pd.set_option('display.min_rows', 50)
pd.set_option('display.max_columns', 50)

In [3]:
os.listdir('z:\\gdrive_backup\\CoderBoy\\Python\\football_stats\\weather_and_location_data\\')

['Bundesliga_venues.parquet',
 'EPL_venues.parquet',
 'Eredivisie_venues.parquet',
 'La_Liga_2_venues.parquet',
 'Ligue_1_venues.parquet',
 'Ligue_2_venues.parquet',
 'Primeira_Liga_venues.parquet',
 'Serie_A_venues.parquet',
 'Serie_B_venue.parquet',
 'venue_locations.ipynb']

In [324]:
serie_a_venues = pd.read_parquet('z:\\gdrive_backup\\CoderBoy\\Python\\football_stats\\weather_and_location_data\\Serie_A_venues.parquet')

In [4]:
[file for file in os.listdir('z:\\gdrive_backup\\CoderBoy\\Python\\football_stats\\matches') if '.pkl' in file]

['Bundesliga 2_FBRef_scrape.pkl',
 'Bundesliga_FBRef_scrape.pkl',
 'EFL Championship_FBRef_scrape.pkl',
 'EPL_FBRef_scrape.pkl',
 'Eredivisie_FBRef_scrape.pkl',
 'La Liga 2_FBRef_scrape.pkl',
 'La Liga_FBRef_scrape.pkl',
 'Ligue 1_FBRef_scrape.pkl',
 'Ligue 2_FBRef_scrape.pkl',
 'missing_matches_FBRef_scrape.pkl',
 'Primeira Liga_FBRef_scrape.pkl',
 'Serie A_FBRef_scrape.pkl',
 'Serie B_FBRef_scrape.pkl']

In [325]:
serie_b_matches = pd.read_pickle(f'Z:\\gdrive_backup\\CoderBoy\\Python\\football_stats\\matches\\Serie B_FBRef_scrape.pkl')
# ligue_2_matches = pd.read_pickle(f'C:\\Users\\Alec\\Documents\\Python\\soccer-ai\\data_files\\matches\\Ligue 2_FBRef_scrape.pkl')
geo_api = 'e96c2545f79147dc882c08a481f7bb74'

In [326]:
serie_b_matches['year'].value_counts().sort_index()

year
2014    471
2015    471
2016    468
2017    460
2018    350
2019    388
2020    387
2021    389
2022    390
2023    150
Name: count, dtype: int64

In [ ]:
bundesliga_2_matches = pd.read_pickle(f'Z:\\gdrive_backup\\CoderBoy\\Python\\football_stats\\matches\\Primeira Liga_FBRef_scrape.pkl')

In [ ]:
bundesliga_2_matches['year'].value_counts().sort_index()

In [ ]:
serie_b_matches = pd.concat([serie_b_matches, bundesliga_2_matches],ignore_index=True)

In [329]:
venue_list = serie_b_matches['venue'].drop_duplicates().to_list()
venue_locations = pd.DataFrame()
len(venue_list)

58

In [330]:
not_found = []
for venue in venue_list:
    url_encoded_venue = urllib.parse.quote(venue)
    base_geo_url = f'https://api.opencagedata.com/geocode/v1/json?q={url_encoded_venue}&key={geo_api}'
    start_time = perf_counter()
    response = requests.get(base_geo_url)
    results = response.json()['results']
    num_results = response.json()['total_results']
    if num_results == 0:
        not_found.append(venue)
    for i in range(num_results):
        lat = results[i]['geometry']['lat']
        lng = results[i]['geometry']['lng']
        name = results[i]['formatted']
        links = results[i]['annotations']['OSM']['url']

        tmp = pd.DataFrame({'venue': venue, 'latitude': lat, 'longitude': lng, 'name': name, 'links': links}, index=[0])
        venue_locations = pd.concat([venue_locations, tmp], ignore_index=True)
    run_time = perf_counter() - start_time
    if run_time < 2:
        time.sleep(2.2 - run_time)

In [331]:
display(response.json()['rate'])
not_found

{'limit': 2500, 'remaining': 2384, 'reset': 1702944000}

[]

In [333]:
keep_indexes = []

In [332]:
venue_locations = venue_locations[~(venue_locations['venue'].isin(serie_a_venues['venue']))].reset_index(drop=True).copy()

In [485]:
len(venue_locations['venue'].drop_duplicates())

34

In [ ]:
serie_b_matches[serie_b_matches['venue'] == venue_locations.loc[i, 'venue']].drop_duplicates('venue')
serie_b_matches[serie_b_matches['venue'].str.lower().str.contains('hardt')].drop_duplicates('venue')

In [ ]:
serie_b_matches[serie_b_matches['home_team'].str.lower().str.contains('hardtwald')].drop_duplicates('venue')

In [ ]:
venue_locations[venue_locations['venue'].str.contains('Hardtwaldstadion')]

In [ ]:
prev_name = venue_locations.loc[w_i, 'venue']
venue_locations.loc[w_i] = venue_locations.loc[c_i].copy()
venue_locations.loc[w_i, 'venue'] = prev_name

In [ ]:
c_i = 96
w_i = 116
test = venue_locations.copy()
prev_name = test.loc[w_i, 'venue']
test.loc[w_i] = test.loc[c_i].copy()
test.loc[w_i, 'venue'] = prev_name
test.loc[w_i:].head()

In [481]:
keep_indexes = list(set(keep_indexes))
display(keep_indexes[-10:])

[41, 42, 44, 46, 47, 49, 51, 52, 54, 56]

In [336]:
not_found

[]

In [460]:
# venue_locations.loc[i, 'venue'] = 'Estádio Nacional, Jamor'
venue_locations.loc[i, 'name'] = "Viale Trieste, 19, 39100 Bolzano BZ, Italy"
venue_locations.loc[i, 'links'] = 'https://www.openstreetmap.org/way/164506172#map=17/46.49177/11.34480'
venue_locations.loc[i, 'latitude'] = 46.49211926165395
venue_locations.loc[i, 'longitude'] = 11.345831262834214

In [ ]:
46.49211926165395, 11.345831262834214

In [480]:
keep_indexes.append(i)
print(keep_indexes[-10:])

[44, 46, 47, 49, 51, 52, 54, 56, 42, 42]


In [474]:
# i = -1
i += 1
display(i)
venue_locations.loc[42, 'links']
# venue_locations.loc[114, 'longitude']

57

'https://www.openstreetmap.org/?mlat=45.14014&mlon=10.03486#map=17/45.14014/10.03486'

In [471]:
venue_locations.loc[i:]

,venue,latitude,longitude,name,links
56,"Stadio Nicola Ceravolo , Catanzaro",38.919706,16.58782,"Stadio Nicola Ceravolo, Via Francesco Paglia, ...",https://www.openstreetmap.org/?mlat=38.91971&m...
57,"Stadio Nicola Ceravolo , Catanzaro",38.882470,16.60086,"Catanzaro, Italy",https://www.openstreetmap.org/?mlat=38.88247&m...


In [482]:
final_venue_locations = venue_locations[venue_locations.index.isin(keep_indexes)].reset_index(drop=True)
final_venue_locations.shape

(34, 5)

In [483]:
venue_locations[~venue_locations['venue'].isin(final_venue_locations['venue'])]

,venue,latitude,longitude,name,links


In [484]:
set(not_found) - set(final_venue_locations['venue']) 

set()

In [ ]:
final_final_venue_locations = pd.concat([final_venue_locations, error_row], ignore_index=True)

In [ ]:
final_final_venue_locations

In [486]:
final_venue_locations.to_parquet('Serie_B_venue.parquet')